In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
memops = {'scal':2, 'axpby':3, 'pointwiseDot':6, 'dot':2, 'dx':3, 'dy':3,
         'arakawa':34, 'cg':42}

In [4]:
#read in csv file
df = pd.read_csv('benchmark_p100_ampi1.csv', delimiter=' ')
#get rid of non-relevant columns
df_measure = df[['n','Nx','Ny']+list(memops.keys())]
#compute show mean and standard derivation of 'same' groups 
dfr=df_measure.groupby(['n', 'Nx','Ny']).agg(['mean', 'std'])
dfr=dfr.reset_index(level=['n','Nx','Ny'])
dfr

n    Nx    Ny        cg                 arakawa                \
                       mean           std      mean           std   
0   2   256   256  0.000365  3.607310e-06  0.000221  2.787494e-06   
1   2   512   512  0.001066  3.195987e-07  0.000803  1.456022e-07   
2   2  1024  1024  0.003778  1.020120e-06  0.003088  1.318627e-07   
3   2  2048  2048  0.014657  3.286944e-06  0.012243  2.485514e-07   
4   3   256   256  0.000731  4.999753e-07  0.000532  1.057303e-07   
5   3   512   512  0.002448  8.622071e-07  0.002006  1.167381e-07   
6   3  1024  1024  0.009313  1.979640e-06  0.007905  2.494795e-07   
7   3  2048  2048  0.036624  2.161275e-06  0.031435  5.872724e-07   
8   4   256   256  0.001293  3.093093e-07  0.001031  9.681598e-08   
9   4   512   512  0.004702  1.311825e-06  0.004013  8.422853e-08   
10  4  1024  1024  0.018417  4.026909e-06  0.016028  3.910101e-07   
11  4  2048  2048  0.072860  2.097644e-06  0.063738  2.915647e-06   
12  6   256   256  0.003361  1.305899e-06  0.002915  1.347261e-07   
13  6   512   512  0.012934  1.474374e-06  0.011519  2.347576e-07   
14  6  1024  1024  0.050825  2.853769e-06  0.045623  2.059234e-06   
15  6  2048  2048       NaN           NaN  0.248301  1.007748e-05   

   pointwiseDot                   axpby                    scal                \
           mean           std      mean           std      mean           std   
0      0.000026  4.332051e-08  0.000005  1.680106e-07  0.000005  1.433725e-07   
1      0.000094  1.825742e-08  0.000047  2.494438e-08  0.000034  4.541170e-08   
2      0.000366  2.558211e-08  0.000183  3.713339e-08  0.000130  5.125102e-08   
3      0.001460  1.220474e-06  0.000725  7.131534e-07  0.000520  9.054213e-07   
4      0.000054  2.043961e-08  0.000028  2.514403e-08  0.000020  3.293090e-08   
5      0.000208  5.945119e-08  0.000104  5.720334e-08  0.000074  3.178050e-08   
6      0.000821  1.384032e-06  0.000410  3.714910e-07  0.000291  3.710481e-07   
7      0.003325  7.633261e-08  0.001633  4.220795e-07  0.001193  7.316921e-07   
8      0.000094  1.911951e-08  0.000047  1.911951e-08  0.000034  3.373096e-08   
9      0.000366  2.424413e-08  0.000183  3.527668e-08  0.000130  3.056868e-08   
10     0.001461  5.161621e-07  0.000725  7.022859e-07  0.000520  8.886975e-07   
11     0.005921  9.829660e-08  0.002917  8.075340e-08  0.002149  1.868114e-06   
12     0.000208  3.368151e-08  0.000104  2.666667e-08  0.000074  4.221637e-08   
13     0.000820  1.648133e-06  0.000409  4.436515e-07  0.000291  4.623323e-07   
14     0.003325  5.542763e-08  0.001633  3.329481e-07  0.001193  2.139173e-06   
15     0.013456  2.011080e-07  0.006602  1.794467e-07  0.004868  9.931230e-07   

          dx                     dot                      dy                
        mean           std      mean           std      mean           std  
0   0.000020  6.371499e-07  0.000482  6.159161e-06  0.000021  6.710233e-07  
1   0.000091  4.948625e-08  0.000519  1.602736e-06  0.000092  3.802046e-08  
2   0.000351  4.516636e-08  0.000659  4.968155e-06  0.000351  5.940445e-08  
3   0.001390  6.416550e-07  0.001237  1.189807e-05  0.001383  6.980131e-08  
4   0.000063  7.601170e-08  0.000497  7.397868e-07  0.000065  4.295993e-08  
5   0.000239  5.437524e-08  0.000578  7.348566e-06  0.000243  2.415229e-08  
6   0.000944  1.237286e-06  0.000907  2.753717e-05  0.000956  1.341194e-06  
7   0.003754  1.841648e-07  0.002199  5.387424e-06  0.003745  1.081614e-07  
8   0.000128  4.321779e-08  0.000515  4.966906e-06  0.000130  1.349897e-08  
9   0.000503  1.131813e-07  0.000649  7.110076e-06  0.000506  3.392803e-08  
10  0.002007  1.131346e-06  0.001235  9.507706e-06  0.002027  1.191871e-07  
11  0.007942  3.712202e-07  0.003538  5.864345e-06  0.008029  1.530142e-06  
12  0.000407  8.071899e-08  0.000581  7.725749e-06  0.000378  6.041523e-08  
13  0.001617  3.476964e-06  0.000905  1.984654e-05  0.001492  1.309244e-06  
14  0.006364  8.193554e-07  0.002200  5.760733e-06  0.005873  1.284307

In [5]:
#insert size in GBytes
dfr.insert(0,'size', 8*dfr['n']*dfr['n']*dfr['Nx']*dfr['Ny']/1e9)
#insert bandwidth columns= multiply size by memops / time
for name, mem in memops.items():
    dfr[(name,'bw')] = dfr['size']*mem/dfr[(name,'mean')]
#now display all bandwidth results

In [6]:
cols=[(m,'bw') for m in memops.keys()]
dfr=dfr[['n','Nx','Ny','size']].join( dfr[cols])
dfr=dfr.sort_values(by='size')
dfr=dfr.set_index('size')

In [7]:
dfr

,n,Nx,Ny,cg,arakawa,pointwiseDot,axpby,scal,dx,dot,dy
,,,,bw,bw,bw,bw,bw,bw,bw,bw
size,,,,,,,,,,,
0.002097,2,256,256,241.617514,322.505973,486.747592,1157.368228,889.943560,320.600082,8.710677,306.377210
0.004719,3,256,256,270.972173,301.545445,520.643496,509.952664,482.030034,224.730529,18.975240,218.530898
0.008389,2,512,512,330.659689,355.073354,535.785054,531.597465,496.778870,277.627518,32.342556,272.584556
0.008389,4,256,256,272.505916,276.726038,535.619704,531.586236,496.455465,196.113121,32.574399,193.099029
0.018874,3,512,512,323.836450,319.957975,544.408428,543.799318,509.422761,236.913779,65.280093,232.983332
0.018874,6,256,256,235.841721,220.127217,544.513134,543.877668,509.333405,139.142251,64.949984,149.667889
0.033554,2,1024,1024,373.070801,369.494209,549.365685,550.623279,514.938645,287.033710,101.784040,286.848858
0.033554,4,512,512,299.698394,284.269962,549.383675,550.644363,515.021634,200.170010,103.381347,198.985724
